# Import Libraries

In [1]:
import random
import tkinter as tk
from PIL import ImageTk
from PIL import Image

# Define Dictionaries

In [2]:
KEY_LIST = [
    ['turn\nleft', 'up', 'turn\nright'],
    ['left', 'down', 'right']
]

KEY_DIC = {
    'turn\nleft': 'tl',
    'turn\nright': 'tr',
    'up': 'u',
    'left': 'l',
    'down': 'd',
    'right': 'r',
}

DIRECTIONS = {
    "U": (-1, 0),
    "D": (1, 0),
    "R": (0, 1),
    "L": (0, -1)
}

ROBOT_TYPE_DIC = {'r': 'red', 'b': 'blue', 'g': 'gray'}

ROBOT_DIRECTION_DIC = {
    'up': Image.ROTATE_90,
    'right': Image.ROTATE_180,
    'down': Image.ROTATE_270,
    'left': None,
}

ROBOT_DIRECTION_LIST = ['up', 'right', 'down', 'left']

ROBOT_TURN = {
    'TL': Image.ROTATE_90,
    'TR': Image.ROTATE_270
}

BODY_DIC = {'Simple': 2, 'Hard': 5, 'Light': 3, 'Battle': 2}

WEAPON_DIC = {'Basic': 2, 'Laser': 5, 'Sword': 3,
              'Explosion': 2, 'Dual': 1, 'Twin': 0}

TOTAL_LIST = ['Simple', 'Hard', 'Light', 'Battle', 'Basic', 'Laser', 'Sword',
              'Explosion', 'Dual', 'Twin', '0', '0', '0', '0', '0']

# Define Functions

In [3]:
def get_not_repeated_random(current, list_type=1, min_count=0, max_count=9):

    # get random numbers

    lists = []
    if list_type == 1:
        while current > 0:
            count = random.randint(min_count, max_count)
            if count not in lists:
                lists.append(count)
                current -= 1
            if current <= 0:
                lists.sort()
                return lists
    else:
        while current > 0:
            count1 = random.randint(min_count + 1, max_count - 1)
            count2 = random.randint(min_count, max_count)
            count_str = f'{count1}_{count2}'

            if count_str not in lists:
                lists.append(count_str)
                current -= 1
            if current <= 0:
                lists.sort()
                return lists


def basic(turn, x, y):
    if turn == 'left':
        xy_list = [[x, y - 1], [x, y - 2]]
    if turn == 'up':
        xy_list = [[x - 1, y], [x - 2, y]]
    if turn == 'down':
        xy_list = [[x + 1, y], [x + 2, y]]
    if turn == 'right':
        xy_list = [[x, y + 1], [x, y + 2]]
    return xy_list


def laser(turn, x, y):
    if turn == 'left':
        xy_list = [[x, y - 1], [x, y - 2], [x, y - 3], [x, y - 4], [x, y - 5]]
    if turn == 'up':
        xy_list = [[x - 1, y], [x - 2, y], [x - 3, y], [x - 4, y], [x - 5, y]]
    if turn == 'down':
        xy_list = [[x + 1, y], [x + 2, y], [x + 3, y], [x + 4, y], [x + 5, y]]
    if turn == 'right':
        xy_list = [[x, y + 1], [x, y + 2], [x, y + 3], [x, y + 4], [x, y + 5]]
    return xy_list


def sword(turn, x, y):
    if turn == 'left':
        xy_list = [[x, y - 1], [x + 1, y - 1], [x - 1, y - 1]]
    if turn == 'up':
        xy_list = [[x - 1, y], [x - 1, y + 1], [x - 1, y - 1]]
    if turn == 'down':
        xy_list = [[x + 1, y], [x + 1, y - 1], [x + 1, y + 1]]
    if turn == 'right':
        xy_list = [[x, y + 1], [x + 1, y + 1], [x - 1, y + 1]]
    return xy_list


def explosion(turn, x, y):
    xy_list = [
        [x, y + 1], [x, y - 1], [x + 1, y], [x - 1, y],
        [x - 1, y + 1], [x + 1, y - 1], [x + 1, y + 1], [x - 1, y - 1]
    ]
    return xy_list


def dual(turn, x, y):
    if turn == 'left' or turn == 'right':
        xy_list = [
            [x + 1, y], [x + 2, y], [x + 3, y], [x + 4, y], [x + 5, y],
            [x - 1, y], [x - 2, y], [x - 3, y], [x - 4, y], [x - 5, y]
        ]
    if turn == 'up' or turn == 'down':
        xy_list = [
            [x, y + 1], [x, y + 2], [x, y + 3], [x, y + 4], [x, y + 5],
            [x, y - 1], [x, y - 2], [x, y - 3], [x, y - 4], [x, y - 5]
        ]
    return xy_list

# Define Abstract Grid

In [4]:
class AbstractGrid(tk.Canvas):
    def __init__(self, master, rows, cols, width, height, **kwargs):
        self._box_width = width / cols
        self._box_height = height / rows
        super().__init__(master, width=width, height=height, **kwargs)

    def get_position_center(self, position):
        row = position[0]
        col = position[1]
        x = self._box_width * col + self._box_width /2
        y = self._box_height * row + self._box_height /2
        return x, y

    def annotate_position(self, position, text):
        xy = self.get_position_center(position)
        self.create_text(xy[0], xy[1], text=text)

    def get_box_width(self):
        return self._box_width

    def get_box_height(self):
        return self._box_height

# Define Direction Keypad

In [5]:
class KeyPad(AbstractGrid):
    
    # define keypad

    def __init__(self, master, width=200, height=100, **kwargs):
        super().__init__(master, 3, 4, width, height, **kwargs)
        self.draw_key_pad()

    def draw_key_pad(self):
        
        # create keypad

        for row in range(len(KEY_LIST)):
            for col in range(len(KEY_LIST[row])):
                if KEY_LIST[row][col] != ' ':
                    xy = self.get_position_center((row, col))
                    key_x = xy[0] - self.get_box_width() / 2
                    key_y = xy[1] - self.get_box_height() / 2
                    self.create_rectangle(key_x + 2, key_y + 2,
                                          self.get_box_width() + key_x - 1,
                                          self.get_box_height() + key_y - 1)
                    self.annotate_position((row, col), KEY_LIST[row][col])

    def pixel_to_direction(self, pixel):
        
        # get the coordinate by clicking

        x = pixel['x']
        y = pixel['y']
        x1 = int(x / self.get_box_width())
        y1 = int(y / self.get_box_height())
        try:
            if y1 <= len(KEY_LIST) and x1 <= len(KEY_LIST[y1]):
                if KEY_LIST[y1][x1] != ' ':
                    return KEY_DIC[KEY_LIST[y1][x1]]
            return None
        except Exception:
            return None

# Instantiate the Robot

In [6]:
class RobotObj(tk.Canvas):
    
    # instantiate robot

    def __init__(self, master, robot_type, w, h):

        self.is_alive = True
        self._master = master
        self._robot_type = robot_type
        self._box_width = int(w) - 6
        self._box_height = int(h) - 6
        self.turn = 'left'
        super().__init__(self._master, width=self._box_width, height=self._box_height)

    def create_bind(self, onCallback):

        self._onCallback = onCallback
        self._label.bind(sequence="<Button-1>", func=self.onRobot)

    def onRobot(self, e):

        self._onCallback(self.get_ij())
        self.selected_type(1)

    def draw_robot(self, ddirection='left'):
        
        # loading the robot

        self._img = Image.open(f'{ROBOT_TYPE_DIC[self._robot_type]}.png').resize(
            (self._box_width, self._box_height), Image.ANTIALIAS)
        dir = ROBOT_DIRECTION_DIC[ddirection]
        if dir is not None:
            self.turn = ddirection
            self._img = self._img.transpose(dir)
        self._pimg = ImageTk.PhotoImage(self._img)
        self._label = tk.Label(self, image=self._pimg)
        self._label.pack()

    def update_ij(self, i, j):
        
        # save the coordinate of robot

        self.__i = i
        self.__j = j

    def get_ij(self):
       
        # get the coordinate of robot
        
        return self.__i, self.__j

    def change_direction(self, ddirection):
        
        # change the direction of robot

        dir = ROBOT_TURN[ddirection]
        if dir is not None:
            self._img = self._img.transpose(dir)
        self._pimg = ImageTk.PhotoImage(self._img)
        self._label['image'] = self._pimg

        v = ROBOT_DIRECTION_LIST.index(self.turn)
        if ddirection == 'TR':
            v += 1
            if v >= len(ROBOT_DIRECTION_LIST):
                v = 0
        else:
            v -= 1
            if v < 0:
                v = len(ROBOT_DIRECTION_LIST) - 1
        self.turn = ROBOT_DIRECTION_LIST[v]

    def boom(self):
        
        # dead effection
        
        self._boom_img = Image.open('dead.png').resize(
            (self._box_width, self._box_height), Image.ANTIALIAS)
        self._boom_pimg = ImageTk.PhotoImage(self._boom_img)
        self._boom_label = tk.Label(self, image=self._boom_pimg)
        self._boom_label.place(x=0, y=0)
        self._boom_label.after(80, lambda: self._boom_label.destroy())

    def des_label(self):
        
        # remove the dead robot 

        self._img = self._pimg = None
        self._label.destroy()

    def drop(self):
        
        # create the items after the gray robot die

        self._drop_img = Image.open('item.png').resize(
            (self._box_width, self._box_height), Image.ANTIALIAS)
        self._drop_pimg = ImageTk.PhotoImage(self._drop_img)
        self._drop_label = tk.Label(self, image=self._drop_pimg)
        self._drop_label.after(80, lambda: self._drop_label.place(x=0, y=0))
        self.is_alive = False

    def des_drop(self):
        
        # remove the item

        self._drop_img = self._drop_pimg = None
        self._drop_label.destroy()
        self.destroy()

    def selected_type(self, select_type=0):
        if self._img != None and self._pimg != None:
            if select_type != 0:
                png_name = f'{ROBOT_TYPE_DIC[self._robot_type]}1.png'
            else:
                png_name = f'{ROBOT_TYPE_DIC[self._robot_type]}.png'
            self._img = Image.open(png_name).resize(
                (self._box_width, self._box_height), Image.ANTIALIAS)
            dir = ROBOT_DIRECTION_DIC[self.turn]
            self._img = self._img.transpose(dir)
            self._pimg = ImageTk.PhotoImage(self._img)
            self._label['image'] = self._pimg

# Define Robot Property

In [7]:
class RobotData():
    
    # set all the property of robot

    def __init__(self, robot_type):
        self.robot_type = robot_type
        if self.robot_type == 'g':
            self.hp = 1
        else:
            self.body = 'Simple'
            self.hp = BODY_DIC.get(self.body)
            self.weapon = 'Basic'
            self.attack = WEAPON_DIC.get(self.weapon)
            self.weapon_slot = 1
            self.weapon_lsit = ['Basic']
            # current movement
            self.c_mvt = 1
            # maximum movement
            self.max_mvt = 1

    def change_hp(self, count):
        self.hp += count

    def change_mvt(self, count):
        self.c_mvt += count

    def change_slot(self, count):
        self.weapon_slot += count

# Define Status Box

In [8]:
class StateBox(tk.Canvas):
    
    # create status bar
    
    __state_txt = '{} Team\n' \
                  'Body Type: {}\n' \
                  'Weapon: {}\n' \
                  'Weapon Slot: {}\n' \
                  'Movement: {}'

    def __init__(self, master, x, y, width, height, border, outline, **kwargs):
        self._x = x
        self._y = y
        self._width = width
        self._height = height
        self._border = border
        self._outline = outline
        self.switch_weapons_btn = None
        super().__init__(master, width=width, height=height, **kwargs)
        self.create_rectangle(
            border, border,
            width - border / 2, height,
            width=border, outline=self._outline
        )
        self.place(x=self._x, y=self._y)

        self._label = tk.Label(self, justify="left", font=('Arial', 12))
        self._label.place(x=20, y=10)

        self._hp_label = tk.Label(self, justify="left", font=('Arial', 24))
        self._hp_label.place(x=20, y=100)

        self.select_label = tk.Label(self, justify='left', font=('Arial', 16))
        self.select_label.place(x=10, y=60)

    def set_label_txt(self, body, weapon, wapon_slot, mvt):
        
        # set information for robot

        self._label['text'] = self.__state_txt.format(self._outline, body, weapon, wapon_slot, mvt)
        if self.select_label is not None:
            self.select_label.destroy()
            self.select_label = None

    def create_switch_weapons_btn(self, on_call_back):
        
        # create the button to change weapon

        # check the button if it's exist
        if self.switch_weapons_btn is None:
            self.switch_weapons_btn = tk.Button(self, text='Switch\nWeapons', command=on_call_back)
            self.switch_weapons_btn.place(x=100, y=80)

    def change_btn_state(self, bo):
        
        # This function is to check if the robot has only one weapon, if it has only one, the button is unclickable

        if bo:
            self.switch_weapons_btn['state'] = 'normal'
        else:
            self.switch_weapons_btn['state'] = 'disabled'

    def set_hp(self, hp):
        
        # set HP for robot

        self._hp_label['text'] = f'HP: {hp}'

    def set_select_txt(self, txt):
        
        # set status bar to show detail information

        self.select_label['text'] = txt

# Define GUI

In [9]:
class GameInit(tk.Canvas):

    def __init__(self, master, rows, cols, **kwargs):
        self._master = master
        self._rows = rows
        self._cols = cols
        # width and height of each grid
        self.box_width = 500 / rows
        self.box_height = 500 / cols
        self.grid_lists = None
        self.robot_lists = None
        # current selected robot
        self.current_robot = None
        # current selected robot data
        self.current_data = None
        # current selected CPU robot
        self.current_enemy_robot = None
        # current selected CPU robot data
        self.current_enemy_data = None
        super().__init__(self._master, width=594, height=594, **kwargs)
        # draw battleground
        self.draw_grid()


    def draw_grid(self):
        
        # create grid

        for i in range(self._rows):
            for j in range(self._cols):
                x = self.box_width * j
                y = self.box_height * i
                
                _color = 'gray'
                self.create_rectangle(
                    x, y,
                    self.box_width + x, self.box_height + y,
                    fill=_color
                )

    def create_grid_data(self, r=5, b=5, g=10):
        
        # create grid data

        # when 0, create robot and display it
        r = random.randint(1, 10) if r == 0 else r
        red_list = get_not_repeated_random(r)
        b = random.randint(1, 10) if b == 0 else b
        blue_list = get_not_repeated_random(b)
        g = random.randint(1, 10) if g == 0 else g
        gray_list = get_not_repeated_random(g, 2)

        # initialize battleground
        grid_list = [[] for x in range(0, 10)]
        for i in range(10):
            for j in range(10):
                # save CPU robot
                if i == 0:
                    if j in red_list:
                        grid_list[i].append(RobotData('r'))
                    else:
                        grid_list[i].append('0')
                # save player robot
                elif i == 9:
                    if j in blue_list:
                        grid_list[i].append(RobotData('b'))
                    else:
                        grid_list[i].append('0')
                # save item robot
                else:
                    count_str = f'{i}_{j}'
                    if count_str in gray_list:
                        grid_list[i].append(RobotData('g'))
                    else:
                        grid_list[i].append('0')
        return grid_list

    def des_game(self):
        
        # delete data after create a new game

        self.grid_lists.clear()
        for i in range(len(self.robot_lists)):
            for j in range(len(self.robot_lists[i])):
                if type(self.robot_lists[i][j]) is RobotObj:
                    self.robot_lists[i][j].destroy()

        self.blue_box.destroy()
        self.red_box.destroy()
        self.l1.destroy()
        self.e1.destroy()
        self.l2.destroy()
        self.e2.destroy()
        self.l3.destroy()
        self.e3.destroy()
        self.n_btn.destroy()
        self.c_btn.destroy()
        self.t1 = None
        self.t2 = None
        self.t3 = None

        self.current_robot = None
        self.current_data = None
        self.current_enemy_robot = None
        self.current_enemy_data = None

    def init_game(self, r=5, b=5, g=5):
        
        # initialize game
        
        # r: CPU robot quantity
        # b: player robot quantity
        # g: item robot quantity
        
        # get data
        self.grid_lists = self.create_grid_data(r, b, g)
        # initialize robot list
        self.robot_lists = [[None for x1 in range(0, 10)] for x2 in range(0, 10)]

        # create robot
        for i in range(len(self.grid_lists)):
            for j in range(len(self.grid_lists[i])):
                r_data = self.grid_lists[i][j]
                if type(r_data) is RobotData:
                    r_box = RobotObj(self, r_data.robot_type, self.box_width, self.box_height)
                    r_box.update_ij(i, j)
                    if r_data.robot_type == 'r':
                        r_box.draw_robot('down')
                        r_box.create_bind(self.on_battle)
                    elif r_data.robot_type == 'b':
                        r_box.draw_robot('up')
                        r_box.create_bind(self.on_battle)
                    elif r_data.robot_type == 'g':
                        if i >= 5:
                            r_box.draw_robot('down')
                        else:
                            r_box.draw_robot('up')
                    r_box.place(x=j * self.box_height + 1, y=i * self.box_width + 1)
                    self.robot_lists[i][j] = r_box

        # player status bar
        self.blue_box = StateBox(self._master, 525, 350, 200, 140, 0, 'Blue')
        self.blue_box.set_select_txt('Player Robot Detail')
        # CPU status bar
        self.red_box = StateBox(self._master, 525, 30, 200, 140, 0, 'Red')
        self.red_box.set_select_txt('Enemy Robot Detail')

        # display add CPU robot textbox
        self.l1 = tk.Label(self._master, text="Red Robot(1-10)")
        self.l1.place(x=0, y=510)
        self.t1 = tk.StringVar()
        self.e1 = tk.Entry(self._master, width=5, textvariable=self.t1)
        self.e1.place(x=120, y=510)

        # display add player robot textbox
        self.l2 = tk.Label(self._master, text="Blue Robot(1-10)")
        self.l2.place(x=0, y=550)
        self.t2 = tk.StringVar()
        self.e2 = tk.Entry(self._master, width=5, textvariable=self.t2)
        self.e2.place(x=120, y=550)

        # display add gray robot textbox
        self.l3 = tk.Label(self._master, text="Gray Robot(1-10)")
        self.l3.place(x=0, y=590)
        self.t3 = tk.StringVar()
        self.e3 = tk.Entry(self._master, width=5, textvariable=self.t3)
        self.e3.place(x=120, y=590)

        # display new game button
        self.n_btn = tk.Button(self._master, text='new game', command=self.on_new_game, width=10, height=2)
        self.n_btn.place(x=200, y=510)
        # display quit button
        self.c_btn = tk.Button(self._master, text='close', command=self.on_close, width=10, height=2)
        self.c_btn.place(x=200, y=570)

    def on_switch(self):
        
        # player change the weapon

        if self.current_data and self.current_data.weapon_slot > 1:
            v = self.current_data.weapon_lsit.index(self.current_data.weapon)
            v += 1
            if v >= len(self.current_data.weapon_lsit):
                v = 0
            self.current_data.weapon = self.current_data.weapon_lsit[v]
            self.update_current()

    def on_over(self):
        
        # end current round

        self.enemy_ai()

    # start a new game
    def on_new_game(self):
        _t1 = self.t1.get()
        _t2 = self.t2.get()
        _t3 = self.t3.get()
        if _t1 != '' and _t1 != '' and _t1 != '':
            if _t1.isdigit() and _t2.isdigit() and _t3.isdigit():
                _t1 = int(_t1)
                _t2 = int(_t2)
                _t3 = int(_t3)
                if _t1 > 0 and _t2 > 0 and _t3 > 0:
                    if _t1 <= 10 and _t2 <= 10 and _t3 <= 10:
                        self.des_game()
                        self.init_game(_t1, _t2, _t3)

    def on_close(self):

        # display close button
        
        self._master.destroy()

    def play(self):
        
        self._kp_view = KeyPad(self._master)
        self._kp_view.place(x=625, y=500)
        self._kp_view.bind(sequence="<Button-1>", func=self.on_key)

        # display attack button
        attack_btn = tk.Button(
            self._master, text='Fire', command=self.on_attack,
            width=10, height=2
        )
        attack_btn.place(x=500, y=510)

        # display end round button
        round_btn = tk.Button(
            self._master, text='End Round', command=self.on_over,
            width=10, height=2
        )
        round_btn.place(x=500, y=570)

    def on_battle(self, ij):
        
        # show robot detail after clicking

        i, j = ij
        _data = self.grid_lists[i][j]
        _box = self.robot_lists[i][j]

        # show CPU detail
        if _data.robot_type == 'r':
            if self.current_enemy_robot and self.current_enemy_robot.is_alive:
                self.current_enemy_robot.selected_type()
            self.current_enemy_data = _data
            self.current_enemy_robot = _box
            self.update_enemy()
        
        # show player detail
        elif _data.robot_type == 'b':
            if self.current_robot and self.current_robot.is_alive:
                self.current_robot.selected_type()
            self.current_data = _data
            self.current_robot = _box
            self.blue_box.create_switch_weapons_btn(self.on_switch)
            self.update_current()

    def update_current(self):

        # update player status bar
        
        self.blue_box.set_label_txt(
            self.current_data.body,
            self.current_data.weapon,
            self.current_data.weapon_slot,
            self.current_data.c_mvt)
        self.blue_box.set_hp(self.current_data.hp)
        self.blue_box.change_btn_state(self.current_data.weapon_slot > 1)

    def update_enemy(self):
        
        # update CPU status bar

        self.red_box.set_label_txt(
            self.current_enemy_data.body,
            self.current_enemy_data.weapon,
            self.current_enemy_data.weapon_slot,
            self.current_enemy_data.c_mvt)
        self.red_box.set_hp(self.current_enemy_data.hp)

    def on_key(self, e):
        
        # turn robot

        click_x = e.x
        click_y = e.y
        keys = self._kp_view.pixel_to_direction({'x': click_x, 'y': click_y})
        if keys:
            keys = keys.upper()
            if self.current_robot is not None:
                if keys == 'TL' or keys == 'TR':
                    self.current_robot.change_direction(keys)
                else:
                    self.on_move(keys)

    def on_move(self, keys):
        
        # move robot

        i, j = self.current_robot.get_ij()

        if self.current_robot is not None:
            # check if there is an obstacle before moving
            dx, dy = self.new_position(self.current_robot, keys)
            if self.is_move(dx, dy) and self.current_data.c_mvt > 0:
                # check if there is a weapon before moving
                _drop_data = self.grid_lists[dx][dy]
                if _drop_data != '0':
                    # get item and check it's weapon or body
                    if _drop_data in BODY_DIC:
                        self.current_data.body = _drop_data
                        self.current_data.hp = BODY_DIC.get(_drop_data)
                        self.current_data.max_mvt = 1
                        if _drop_data == 'Light':
                            self.current_data.max_mvt = 2
                            if self.current_data.c_mvt < self.current_data.max_mvt:
                                self.current_data.change_mvt(1)
                        elif _drop_data == 'Battle':
                            self.current_data.change_slot(1)
                    elif _drop_data in WEAPON_DIC:
                        if _drop_data == 'Twin':
                            self.current_data.change_slot(2)
                        else:
                            if self.current_data.weapon_slot > 1:
                                if _drop_data not in self.current_data.weapon_lsit:
                                    self.current_data.weapon_lsit.append(_drop_data)
                            else:
                                self.current_data.weapon = _drop_data
                                self.current_data.attack = WEAPON_DIC.get(_drop_data)
                    _drop_box = self.robot_lists[dx][dy]
                    if not _drop_box.is_alive:
                        _drop_box.des_drop()

                # move robot
                self.robot_lists[dx][dy] = self.current_robot
                self.robot_lists[i][j] = None
                self.current_robot.update_ij(dx, dy)
                self.current_robot.place(x=dy * self.box_height + 1, y=dx * self.box_width + 1)

                # update
                self.grid_lists[dx][dy] = self.current_data
                self.grid_lists[i][j] = '0'
                self.current_data.change_mvt(-1)
                
                # update display
                self.update_current()

    def new_position(self, robot, keys):
        
        # get new coordinate after moving

        x, y = robot.get_ij()
        dx, dy = DIRECTIONS[keys]
        return x + dx, y + dy

    def is_move(self, dx, dy):
        
        # check the direction to move

        if dx < 0 or dy < 0:
            return False
        if dx >= len(self.grid_lists) or dy >= len(self.grid_lists[dx]):
            return False
        if type(self.grid_lists[dx][dy]) is RobotData and self.robot_lists[dx][dy].is_alive:
            return False
        return True

    def on_attack(self):
        
        # control robot to attack

        if self.current_robot is None or self.current_data is None:
            return
        weapon_type = self.current_data.weapon
        weapon_attack = self.current_data.attack
        x, y = self.current_robot.get_ij()
        turn = self.current_robot.turn
        if weapon_type == 'Basic':
            xy_list = basic(turn, x, y)
        elif weapon_type == 'Laser':
            xy_list = laser(turn, x, y)
        elif weapon_type == 'Sword':
            xy_list = sword(turn, x, y)
        elif weapon_type == 'Explosion':
            xy_list = explosion(turn, x, y)
        elif weapon_type == 'Dual':
            xy_list = dual(turn, x, y)

        _data_box_list = self.is_attack(xy_list, self.current_data.robot_type)
        if _data_box_list is None:
            return

        for _data, _box in _data_box_list:
            # show dead effection
            _box.boom()
            _data.hp -= weapon_attack
            if _data.hp <= 0:
                _data.hp = 0
                _box.des_label()
                i, j = _box.get_ij()
                # drop item
                _drop = TOTAL_LIST[random.randint(0, len(TOTAL_LIST) - 1)]
                self.grid_lists[i][j] = _drop
                if _drop != '0':
                    _box.drop()
                else:
                    _box.destroy()
            if self.current_enemy_robot and _box == self.current_enemy_robot:
                self.update_enemy()

        _result_type = self.result_type()
        if _result_type == 1:
            self.show_result()
        elif _result_type == 2:
            self.show_result('lose')

    def is_attack(self, xy_list, data_type):
        
        # check the direction to attack
        # xy_list: target coordinate

        _data_box_list = []
        for xy in xy_list:
            dx = xy[0]
            dy = xy[1]
            if dx < 0 or dy < 0:
                continue
            if dx >= len(self.grid_lists) or dy >= len(self.grid_lists[dx]):
                continue

            if type(self.grid_lists[dx][dy]) is RobotData and self.robot_lists[dx][dy].is_alive:
                _data = self.grid_lists[dx][dy]
                _box = self.robot_lists[dx][dy]
                if _data.robot_type != data_type:
                    _data_box_list.append([_data, _box])
        return _data_box_list

    def next_round(self):
        
        # end this round and initialize movement of robot

        for i in range(len(self.grid_lists)):
            for j in range(len(self.grid_lists[i])):
                _data = self.grid_lists[i][j]
                if type(_data) is RobotData:
                    if _data.robot_type == 'r' or _data.robot_type == 'b':
                        _data.c_mvt = _data.max_mvt
                    if self.current_enemy_data and self.current_enemy_data == _data:
                        self.update_enemy()
                    if self.current_data and self.current_data == _data:
                        self.update_current()

    def enemy_ai(self):
        
        # actions of CPU
        
        _result_type = 0
        for i in range(len(self.grid_lists)):
            for j in range(len(self.grid_lists[i])):
                _data = self.grid_lists[i][j]
                _robot = self.robot_lists[i][j]
                if type(_data) is RobotData and _data.robot_type == 'r' and _robot.is_alive:
                    move_times = 10
                    while _data.c_mvt >= 1:
                        if _result_type == 2:
                            return
                        move_times -= 1
                        if move_times <= 0:
                            break
                        # check if there is an obstacle before moving
                        kk = ROBOT_DIRECTION_LIST[random.randint(0, len(ROBOT_DIRECTION_LIST) - 1)]
                        dx, dy = self.new_position(_robot, KEY_DIC[kk].upper())
                        if self.is_move(dx, dy):
                            _drop_data = self.grid_lists[dx][dy]
                            if _drop_data != '0':
                                # get item and check it's weapon or body
                                if _drop_data in BODY_DIC:
                                    _data.body = _drop_data
                                    _data.hp = BODY_DIC.get(_drop_data)
                                    _data.max_mvt = 1
                                    if _drop_data == 'Light':
                                        _data.max_mvt = 2
                                        if _data.c_mvt < _data.max_mvt:
                                            _data.change_mvt(1)
                                    elif _drop_data == 'Battle':
                                        _data.change_slot(1)
                                elif _drop_data in WEAPON_DIC:
                                    if _drop_data == 'Twin':
                                        _data.change_slot(2)
                                    else:
                                        if _data.weapon_slot > 1:
                                            if _drop_data not in _data.weapon_lsit:
                                                _data.weapon_lsit.append(_drop_data)
                                        else:
                                            _data.weapon = _drop_data
                                            _data.attack = WEAPON_DIC.get(_drop_data)
                                _drop_box = self.robot_lists[dx][dy]
                                if not _drop_box.is_alive:
                                    _drop_box.des_drop()

                            # move the robot
                            self.robot_lists[dx][dy] = _robot
                            self.robot_lists[i][j] = None
                            _robot.update_ij(dx, dy)
                            _robot.place(x=dy * self.box_height + 1, y=dx * self.box_width + 1)

                            # update robot
                            self.grid_lists[dx][dy] = _data
                            self.grid_lists[i][j] = '0'
                            _data.change_mvt(-1)
                            
                            # update display
                            if self.current_enemy_data and self.current_enemy_data == _data:
                                self.update_enemy()

                        weapon_type = _data.weapon
                        weapon_attack = _data.attack
                        x, y = _robot.get_ij()
                        turn = _robot.turn
                        if weapon_type == 'Basic':
                            xy_list = basic(turn, x, y)
                        elif weapon_type == 'Laser':
                            xy_list = laser(turn, x, y)
                        elif weapon_type == 'Sword':
                            xy_list = sword(turn, x, y)
                        elif weapon_type == 'Explosion':
                            xy_list = explosion(turn, x, y)
                        elif weapon_type == 'Dual':
                            xy_list = dual(turn, x, y)

                        _data_box_list = self.is_attack(xy_list, _data.robot_type)
                        if _data_box_list is None:
                            return

                        for _data1, _box1 in _data_box_list:
                            # show dead picture
                            _box1.boom()
                            _data1.hp -= weapon_attack
                            if _data1.hp <= 0:
                                _data1.hp = 0
                                # drop the item if robot dead
                                _box1.des_label()
                                ii, jj = _box1.get_ij()
                                _drop = TOTAL_LIST[random.randint(0, len(TOTAL_LIST) - 1)]
                                self.grid_lists[ii][jj] = _drop
                                if _drop != '0':
                                    _box1.drop()
                                else:
                                    _box1.destroy()
                            if self.current_robot and _box1 == self.current_robot:
                                self.update_current()

                        _result_type = self.result_type()
                        if _result_type == 1:
                            self.show_result()
                        elif _result_type == 2:
                            self.show_result('lose')
        self.next_round()

    def result_type(self):
        
        # check win

        # 0: continue gaming 
        # 1: player win 
        # 2: CPU win
        
        _type = 0
        # check if there is any blue robot remaining
        _have_blue = False
        # check if there is any red robot remaining
        _have_red = False

        for i in range(len(self.grid_lists)):
            for j in range(len(self.grid_lists[i])):
                _data = self.grid_lists[i][j]
                if type(_data) is RobotData:
                    if _data.robot_type == 'r':
                        _have_red = True
                    elif _data.robot_type == 'b':
                        _have_blue = True
                    if _have_blue and _have_red:
                        return _type
        if _have_blue:
            _type = 1
        elif _have_red:
            _type = 2
        return _type

    def show_result(self, result_type='win'):
        
        # display game result

        top_level = tk.Toplevel()
        top_level.title("Battle Result")
        tmpcnf = '%dx%d+%d+%d' % (300, 90, 350, 300)
        top_level.geometry(tmpcnf)

        result_txt = 'You Win!!!'
        if result_type == 'lose':
            result_txt = 'You Lose!'
        tk.Label(top_level, text=result_txt, font=('Arial', 18), width=300).pack()
        tk.Button(top_level, text='Done', command=top_level.destroy).pack(side=tk.BOTTOM)


# Define Main Function

In [10]:
def main():
    root = tk.Tk()
    root.geometry('800x650')
    root.title('Super Robot Game')

    center = tk.Canvas(root, width=800, height=800)
    center.place(x=0, y=0)
    center.create_rectangle(6, 6, 506, 506, width=4)
    center.create_rectangle(6, 516, 506, 206, width=6)

    games = GameInit(root, 10, 10)
    games.init_game()
    games.play()
    games.place(x=1, y=1)

    root.mainloop()
    
if __name__ == "__main__":
    main()
